# Ver1. origin rating data 활용
---
## sparsity 고려하지 않고 origin rating data를 이용해 LightFM 모델을 학습해봅니다.


In [1]:
import os
import pandas as pd
import numpy as  np

In [2]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\Whisky\\preprocessing'

In [3]:
rating = pd.read_csv("../dataset/rating.csv", index_col = 0, encoding="utf-8")

In [4]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908176 entries, 0 to 11258
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   username     908176 non-null  object 
 1   whisky_name  908176 non-null  object 
 2   rating       888132 non-null  float64
dtypes: float64(1), object(2)
memory usage: 27.7+ MB


## 1. rating 결측치 처리
---
- 약  900K 건으로 데이터가 여유롭지 않으니 평균으로 대체해줍니다.

In [5]:
# Reset the index of the DataFrame
rating = rating.reset_index(drop=True)

# Group by "whisky_name" and fill missing values in "rating" with the rounded mean of that group
rating["rating"] = rating.groupby("whisky_name")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3124\2115351171.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rating["rating"] = rating.groupby("whisky_name")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))


In [6]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908176 entries, 0 to 908175
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   username     908176 non-null  object 
 1   whisky_name  908176 non-null  object 
 2   rating       908176 non-null  float64
dtypes: float64(1), object(2)
memory usage: 20.8+ MB


- 정상적으로 rating에 각 위스키에 대한 평균 데이터로 대체되었음을 확인할 수 있습니다.

## rating data에서 username, whisky_name 을 index로 변경합니다.

In [7]:
whisky = pd.read_csv("../dataset/whisky.csv", index_col=0, encoding="utf-8")

In [8]:
print(rating['whisky_name'].nunique())
print(whisky['name'].nunique())

3466
3535


### create whisky2Idx

In [9]:
whisky2Idx = {}
for i, whisky_name in enumerate(whisky['name'].unique()):
    whisky2Idx[whisky_name] = i

whisky2Idx

{'Hibiki 21 Year': 0,
 'Highland Park 18 Year': 1,
 "Michter's 20 Year Kentucky Straight Bourbon (2021 Release)": 2,
 'George T. Stagg Bourbon (Fall 2019)': 3,
 'Bowmore Mizunara Cask Finish': 4,
 'Macallan M': 5,
 'Port Ellen 32 Year 1979 (2012 Special Release)': 6,
 'Thomas H. Handy Sazerac Rye (Fall 2015)': 7,
 'William Larue Weller Bourbon (Fall 2015)': 8,
 'Balvenie Tun 1509 Batch 1': 9,
 'Booker\'s Bourbon Batch 2015-01 "Big Man, Small Batch"': 10,
 'Four Roses Limited Edition Single Barrel Bourbon (2013)': 11,
 'Macallan Rare Cask': 12,
 'Four Roses Limited Edition Single Barrel Bourbon (2014)': 13,
 'Macallan Fine Oak 21 Year': 14,
 'Thomas H. Handy Sazerac Rye (Fall 2014)': 15,
 "Parker's Heritage Promise of Hope": 16,
 'George Dickel x Leopold Bros Collaboration Blend (2021 Release)': 17,
 'Booker\'s Bourbon Batch 2021-01 "Donohoe\'s Batch"': 18,
 'Bushmills 28 Year The Rare Casks Cognac Cask Single Malt': 19,
 'Bunnahabhain 2008 Mòine Bordeaux Red Wine Cask Matured': 20,
 "B

### create user2Idx

In [10]:
user2Idx = {}
for i, username in enumerate(rating['username'].unique()):
    user2Idx[username] = i

print(len(user2Idx))
user2Idx

119515


{'AngusBennion': 0,
 'ANTHONY-DIGIROLAMO': 1,
 'Heejin-Lim': 2,
 'whisky_lover': 3,
 'WhiskeyThreePutt': 4,
 'quarterback': 5,
 'Matthew-Wilson2': 6,
 'Paul_Bruce': 7,
 'Josh-Witmer': 8,
 'smitaah': 9,
 'pmihlek': 10,
 'eyepecan': 11,
 'Youbin-Kim': 12,
 'jaredss07': 13,
 'hatmoy': 14,
 'Yaroslav_Prod': 15,
 'coffeestain': 16,
 'El_Flaco': 17,
 'Navfox': 18,
 'Unknown': 19,
 'qatest1': 20,
 'Anders17': 21,
 'jschruef': 22,
 'lethinop': 23,
 'Sylvo91': 24,
 'Vulturecap': 25,
 'mezcalandwhiskey': 26,
 'Benjamin-Fortier': 27,
 'ChiefNoir': 28,
 'Cheyenne-Sonntag': 29,
 'Reebie29': 30,
 'Melissa-Frangi': 31,
 'rwg32': 32,
 'Jas0021': 33,
 'crom760': 34,
 'Room13': 35,
 'torgex': 36,
 'Tasha-Loven': 37,
 'lagavulin25': 38,
 '1840': 39,
 'jimmyku': 40,
 'Chaos9110': 41,
 'oalfon': 42,
 'whiskyronin': 43,
 'Anar-Bt': 44,
 'luoshao': 45,
 'cclambdas': 46,
 'TheIronMonkey': 47,
 'KStasch': 48,
 'yahwawa': 49,
 'joedorn': 50,
 'MNAmateur': 51,
 'cuongtr': 52,
 'shizaam': 53,
 'chitarrista35': 54

### user2Idx, whisky2Idx file로 저장

In [11]:
user_list = list(user2Idx.items())
columns = ["username", "user_id"]

In [12]:
user_id = pd.DataFrame(user_list, columns=columns)
user_id = user_id.reindex(columns=["user_id", "username"])
user_id

,user_id,username
0,0,AngusBennion
1,1,ANTHONY-DIGIROLAMO
2,2,Heejin-Lim
3,3,whisky_lover
4,4,WhiskeyThreePutt
...,...,...
119510,119510,Hyppolite-Pierre-Jr
119511,119511,Vladim-r-apek
119512,119512,virglynn1
119513,119513,Will-Rosenfeld


In [13]:
whisky_list = list(whisky2Idx.items())
columns = ["whisky_name", "whisky_id"]

In [14]:
whisky_id = pd.DataFrame(whisky_list, columns=columns)
whisky_id = whisky_id.reindex(columns=["whisky_id", "whisky_name"])
whisky_id

,whisky_id,whisky_name
0,0,Hibiki 21 Year
1,1,Highland Park 18 Year
2,2,Michter's 20 Year Kentucky Straight Bourbon (2...
3,3,George T. Stagg Bourbon (Fall 2019)
4,4,Bowmore Mizunara Cask Finish
...,...,...
3530,3530,Wild Turkey Spiced
3531,3531,Seagram's 7 Crown American Blended Whiskey
3532,3532,11 Wells Single Malt Whiskey
3533,3533,Immortal Spirits Early Whiskey


In [15]:
whisky_id.loc[whisky_id["whisky_name"].str.contains('Lagavulin')]

,whisky_id,whisky_name
95,95,Lagavulin Distillers Edition
123,123,Lagavulin 12 Year (2017 Special Release)
129,129,Lagavulin 25 Year
131,131,Lagavulin Jazz Festival 2015
220,220,Lagavulin Fèis Ìle 2015
279,279,Lagavulin 26 Year (2021 Special Release)
371,371,Lagavulin 21 Year (2012 Special Release)
416,416,Lagavulin 12 Year The Flames of the Phoenix (2...
517,517,Lagavulin 12 Year (2016 Special Release)
609,609,Lagavulin 16 Year


In [17]:
whisky_id.loc[whisky_id["whisky_name"] == 'Lagavulin Fèis Ìle 2015']

,whisky_id,whisky_name
220,220,Lagavulin Fèis Ìle 2015


In [18]:
rating

,username,whisky_name,rating
0,AngusBennion,Hibiki 21 Year,9.0
1,ANTHONY-DIGIROLAMO,Hibiki 21 Year,7.0
2,Heejin-Lim,Hibiki 21 Year,9.0
3,whisky_lover,Hibiki 21 Year,10.0
4,WhiskeyThreePutt,Hibiki 21 Year,9.0
...,...,...,...
908171,Cptndrinksalot,Triple Crown Blended Whiskey,4.0
908172,Will-Rosenfeld,Triple Crown Blended Whiskey,2.0
908173,Conrad-Hasper,Triple Crown Blended Whiskey,8.0
908174,LRScotchDude,Triple Crown Blended Whiskey,4.0


In [19]:
print(user2Idx["Will-Rosenfeld"])
print(whisky2Idx["Triple Crown Blended Whiskey"])

119513
3534


In [20]:
rating['user_id'] = rating['username'].map(user2Idx)
rating['whisky_id'] = rating['whisky_name'].map(whisky2Idx)

# Drop the old username and whisky_name columns
new_rating = rating.drop(['username', 'whisky_name'], axis=1)
new_rating = new_rating.reindex(columns=["user_id", "whisky_id", "rating"])

In [21]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908176 entries, 0 to 908175
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   username     908176 non-null  object 
 1   whisky_name  908176 non-null  object 
 2   rating       908176 non-null  float64
 3   user_id      908176 non-null  int64  
 4   whisky_id    903797 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 34.6+ MB


In [ ]:
908176 - 903797

In [ ]:
rating.loc[rating['whisky_id'].isnull()]

In [23]:
latin_whisky = rating.loc[rating['whisky_id'].isnull()]["whisky_name"].unique()

In [25]:
print(len(latin_whisky))
latin_whisky


51


array(['Bunnahabhain 2008 MÃ²ine Bordeaux Red Wine Cask Matured',
       'Lagavulin FÃ¨is Ã\x8cle 2015',
       'SÃ¤ntis Malt Appenzeller Single Malt Edition Dreifaltigkeit',
       'Bruichladdich Laddie Origins FÃ¨is Ã\x8cle 2021',
       'Bowmore 15 Year FÃ¨is Ã\x8cle 2019',
       'Caol Ila 22 Year FÃ¨is Ã\x8cle 2019',
       'WhistlePig The Boss Hog ?? The Samurai Scientist',
       "Parker's Heritage Barrel Finished in Orange CuraÃ§ao Barrels",
       'Port Charlotte PC11 EÃ²rna Na h-Alba', 'Waterford The CuvÃ©e',
       'Belle Meade Bourbon MourvÃ¨dre Cask Finish', 'Mackmyra GrÃ¶nt Te',
       'Ledaig 1998 Pedro XimÃ©nez Sherry Cask Finish',
       'Glenmorangie SpÃ¬os', 'Mackmyra Moment BÃ¤rnsten',
       'Bunnahabhain CeÃ²banach',
       'Laphroaig CÃ\xa0irdeas 2019 Triple Wood Cask Strength',
       'High Coast DÃ¡lvve Sherry Influence',
       'Green Spot ChÃ¢teau LÃ©oville Barton Single Pot Still',
       "TÃ© Bheag Connoisseurs' Blended Whisky",
       'Glenlivet NÃ\xa0durr

In [ ]:
rating.iloc[853883]

In [ ]:
rating['whisky_id'] = rating['whisky_id'].astype(int)

In [ ]:
user_id.to_csv("../dataset/v1/user_id.csv")
whisky_id.to_csv("../dataset/v1/whisky_id.csv")
rating.to_csv("../dataset/v1/rating.csv")